# Tavily Web Search
- LLM에게 웹검색을 쥐어주자

In [ ]:
%pip install -q langchain-tavily

In [6]:
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

True

In [7]:
from langchain_tavily import TavilySearch   

search_tool = TavilySearch(
    max_results=5,
    topic="general",
    search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [8]:
pprint(search_tool.invoke({'query': '강남 신세계에서 밥먹을만한 곳 알려줘'}))

{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '강남 신세계에서 밥먹을만한 곳 알려줘',
 'request_id': 'd841f09c-0962-41dd-812c-a5e6982c9941',
 'response_time': 0.91,
 'results': [{'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.71948266,
              'title': '강남 신세계 푸드코트 하우스오브 신세계,스위트파크(Eng CC)/ Seoul ...',
              'url': 'https://www.youtube.com/watch?v=wRvUnu8OwWI'},
             {'content': "'강남신세계백화점 밥' 맛집 연남토마 고속터미널점(덮밥, ☆3.9), 로봇김밥 "
                         '강남고속터미널점(김밥, ☆3.7), 소녀방앗간 서울고속터미널점(산나물밥, ☆4.3) 등 '
                         '80곳의',
              'raw_content': None,
              'score': 0.7152057,
              'title': "'강남신세계백화점 밥' 맛집 빅데이터 추천순위 Top80 - 다이닝코드",
              'url': 'https://www.diningcode.com/list.dc?query=%EA%B0%95%EB%82%A8%EC%8B%A0%EC%84%B8%EA%B3%84%EB%B0%B1%ED%99%94%EC%A0%90+%EB%B0%A5'},
             {'content': 'Missing: 밥 알려줘',
              'raw_content': None,
              'score': 0.6

In [9]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(model='gpt-4.1-nano')

prompt = PromptTemplate(
    input_variables=['query', 'search_results'],
    template="""넌 웹 검색결과를 요약해서 사용자의 질문에 맞게 좋은 답을 해주는 챗봇이야.

    질문: {query}
    검색결과: {search_results}
    질문에 맞는 답을 검색결과 바탕으로 간단하게 알려줘
    """
)

chain = (
    {
        'query': RunnablePassthrough(),  # 사용자 입력 그대로 이 자리에 들어옴
        'search_results': search_tool | RunnableLambda(
            lambda x: '\n'.join(
                [f'-{r['title']} ({r['content']})' for r in x['results']]
            )
        )
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('강남 신세계에서 점심 먹기 좋은곳')

'강남 신세계 백화점 내에서 점심을 맛있게 드시려면, 타치바나(한국요리), 포비 강남점, 베통 신세계백화점 강남점, 일상정원 센트럴시티점 등 다양한 추천 맛집이 있습니다. 특히, 신세계백화점 지하 1층에 위치한 타치바나는 일본 요리와 퓨전 요리를 즐기실 수 있어 좋습니다. 간단하게 식사하실 경우, 101번지남산돈까스(돈까스), 한우리 센트럴시티점(한우 국수전골), 모스버거 센트럴시티점(버거)도 추천드립니다.'

In [10]:
dumb_chain = prompt | llm | StrOutputParser()

dumb_chain.invoke({'query': '강남 신세계에서 점심먹기 좋은곳', 'search_results': ''})

'강남 신세계 백화점 내에서는 다양한 레스토랑과 카페가 있어 점심시간에 적합한 곳이 많습니다. 예를 들면, 한식을 원하시면 한식당이나 퓨전 한식을, 일식을 원하시면 일식 레스토랑을 추천드릴 수 있습니다. 특히, 신세계 백화점 10층이나 11층에 위치한 푸드코트와 레스토랑이 편리하고 맛도 좋아 점심 식사하기에 좋습니다.'

In [11]:
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime

today = datetime.today().strftime('%D')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

prompt = ChatPromptTemplate.from_messages([
    ('system', f'너는 훌륭한 어시스턴트야. 반드시 웹 검색도구를 사용해서 정보를 얻어. 최대한 답을 잘 해보자. 오늘은 {today}야.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, memory=memory, tools=[search_tool], verbose=True)



C:\Users\user\AppData\Local\Temp\ipykernel_13128\2464979423.py:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [12]:
agent_executor.invoke({'input': '아냐 나는 승은이야'})



> Entering new AgentExecutor chain...
반가워, 승은이! 어떻게 도와줄까?

> Finished chain.


{'input': '아냐 나는 승은이야',
 'chat_history': [HumanMessage(content='아냐 나는 승은이야', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반가워, 승은이! 어떻게 도와줄까?', additional_kwargs={}, response_metadata={})],
 'output': '반가워, 승은이! 어떻게 도와줄까?'}